In [ ]:
!pip install flask
!pip install flask-cors
!pip install opencv-python
!pip install numpy
!pip install tensorflow

In [ ]:
!pip install mediapipe==0.10.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.17.0 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
!pip install pyngrok

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving best_model.h5 to best_model.h5


In [ ]:
!lsof -i :5000

COMMAND   PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python3 25396 root    4u  IPv4 642073      0t0  TCP localhost:5000->localhost:46888 (ESTABLISHED)
python3 25396 root    5u  IPv4 625128      0t0  TCP *:5000 (LISTEN)
ngrok   25429 root    6u  IPv4 642072      0t0  TCP localhost:46888->localhost:5000 (ESTABLISHED)


In [ ]:
!kill -9 25396

In [ ]:
!pip install flask-ngrok
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [ ]:
%%writefile app.py
from flask import Flask, request, jsonify
from flask_cors import CORS
import cv2
import mediapipe as mp
import numpy as np
import math
import os
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import subprocess

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

UPLOAD_FOLDER = 'uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def convert_webm_to_mp4(input_path, output_path):
    try:
        subprocess.run(
            ['ffmpeg', '-y', '-i', input_path, output_path],
            check=True,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        print("[INFO] Video successfully converted to MP4.")
        return True
    except subprocess.CalledProcessError:
        print("[ERROR] Conversion to MP4 failed.")
        return False

print("[INFO] Loading model...")
model_path = "best_model.h5"
model = load_model(model_path)
print("[INFO] Model loaded successfully.")

current_status = "focused"
avg_pred = 0

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

def fuzzify(pred):
    print(f"[DEBUG] Fuzzifying prediction value: {pred}")
    if pred == 0.265:
        return {'drowsy': 0, 'focused': 0, 'slightly_drowsy': 1}
    drowsy = max(0, min(1, (0.265 - pred) / 0.05))
    not_drowsy = max(0, min(1, (pred - 0.2651) / 0.05))
    print(f"[DEBUG] Fuzzy values - Drowsy: {drowsy}, Focused: {not_drowsy}")
    return {'drowsy': drowsy, 'focused': not_drowsy}

def preprocess_eye(eye_input):
    img = cv2.cvtColor(eye_input, cv2.COLOR_BGR2GRAY)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=-1)
    return img

def get_eye_crop(landmarks, indices, w, h, image):
    points = [(int(landmarks[i].x * w), int(landmarks[i].y * h)) for i in indices]
    x_coords, y_coords = zip(*points)
    x_min, x_max = max(min(x_coords) - 5, 0), min(max(x_coords) + 5, w)
    y_min, y_max = max(min(y_coords) - 5, 0), min(max(y_coords) + 5, h)
    return image[y_min:y_max, x_min:x_max]

def rotation_vector_to_euler_angles(rotation_vector):
    rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
    print(f"[DEBUG] Rotation Matrix:\n{rotation_matrix}")

    sy = math.sqrt(rotation_matrix[0, 0] ** 2 + rotation_matrix[1, 0] ** 2)

    singular = sy < 1e-6

    if not singular:
        x = math.atan2(rotation_matrix[2, 1], rotation_matrix[2, 2])
        y = math.atan2(-rotation_matrix[2, 0], sy)
        z = math.atan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
    else:
        x = math.atan2(-rotation_matrix[1, 2], rotation_matrix[1, 1])
        y = math.atan2(-rotation_matrix[2, 0], sy)
        z = 0

    return np.degrees([x, y, z])  # pitch, yaw, roll

def estimate_head_pose(landmarks, w, h):
    print("[DEBUG] Estimating head pose...")
    image_points = np.array([
        (landmarks[1].x * w, landmarks[1].y * h),
        (landmarks[152].x * w, landmarks[152].y * h),
        (landmarks[33].x * w, landmarks[33].y * h),
        (landmarks[263].x * w, landmarks[263].y * h),
        (landmarks[61].x * w, landmarks[61].y * h),
        (landmarks[291].x * w, landmarks[291].y * h)
    ], dtype="double")

    model_points = np.array([
        (0.0, 0.0, 0.0),
        (0.0, -63.6, -12.5),
        (-43.3, 32.7, -26),
        (43.3, 32.7, -26),
        (-28.9, -28.9, -24.1),
        (28.9, -28.9, -24.1)
    ])

    focal_length = w
    center = (w / 2, h / 2)
    camera_matrix = np.array([
        [focal_length, 0, center[0]],
        [0, focal_length, center[1]],
        [0, 0, 1]
    ], dtype="double")

    dist_coeffs = np.zeros((4, 1))

    success, rotation_vector, translation_vector = cv2.solvePnP(
        model_points, image_points, camera_matrix, dist_coeffs
    )

    if not success:
        raise ValueError("Head pose estimation failed with solvePnP.")

    print("[DEBUG] Head pose estimated.")
    return rotation_vector, translation_vector


def process_video(video_path):
    global current_status, avg_pred
    print(f"[INFO] Processing video: {video_path}")
    preds = []
    distraction_count = 0
    valid_frame_count = 0

    print("[INFO] Processing video:", video_path)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("[ERROR]Failed to open video.")
        return

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f"[DEBUG] Frame count: {frame_count}, FPS: {fps}")

    if frame_count <= 0:
        print("[ERROR] Video has no frames.")
        current_status = "focused"
        avg_pred = 0
        return

    selected_frames = np.linspace(0, frame_count - 1, 10, dtype=int)
    print(f"[DEBUG] Selected frames: {selected_frames}")

    current_frame = 0
    while current_frame < frame_count:
        ret, frame = cap.read()
        if not ret or frame is None:
            print(f"[ERROR] Frame {current_frame} could not be read. Skipping.")
            current_frame += 1
            continue

        if current_frame in selected_frames:
            print(f"[INFO] Processing frame {current_frame}")
            h, w, _ = frame.shape
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = face_mesh.process(rgb)

            if result.multi_face_landmarks:
                for face_landmarks in result.multi_face_landmarks:
                    print("[DEBUG] Estimating head pose...")
                    rotation_vector, translation_vector = estimate_head_pose(face_landmarks.landmark, w, h)
                    print("[DEBUG] Head pose estimated.")
                    pitch, yaw, roll = rotation_vector_to_euler_angles(rotation_vector)
                    pitch = (pitch) % 360 - 180

                    print(f"[DEBUG] Pitch: {pitch:.2f}, Yaw: {yaw:.2f}, Roll: {roll:.2f}")


                    if abs(yaw) > 15 or abs(pitch) > 15:
                        print("[WARNING] Head pose indicates distraction.")
                        current_status = "distracted"
                        avg_pred = 0
                        distraction_count += 1
                        continue

                    left_eye_idx = [362, 385, 387, 263, 373, 380]
                    right_eye_idx = [33, 160, 158, 133, 153, 144]

                    left_eye = get_eye_crop(face_landmarks.landmark, left_eye_idx, w, h, frame)
                    right_eye = get_eye_crop(face_landmarks.landmark, right_eye_idx, w, h, frame)

                    if left_eye.size == 0 or right_eye.size == 0:
                        print("[WARNING] Eye crop failed. Skipping frame.")
                        continue

                    left_eye = cv2.resize(cv2.flip(left_eye, 1), (64, 64))
                    right_eye = cv2.resize(right_eye, (64, 64))

                    left_proc = preprocess_eye(left_eye)
                    right_proc = preprocess_eye(right_eye)

                    valid_frame_count += 1

                    pred_l = model.predict(left_proc)[0][0]
                    pred_r = model.predict(right_proc)[0][0]
                    print(f"[DEBUG] Predictions - Left: {pred_l:.4f}, Right: {pred_r:.4f}")

                    preds.append((pred_l + pred_r) / 2)


        current_frame += 1

    cap.release()

    if distraction_count > valid_frame_count / 2:
        current_status = "distracted"
        avg_pred = 0
        print("[INFO] Classification: Distracted")
    elif preds:
        avg_pred = sum(preds) / len(preds)
        avg_pred = round(avg_pred, 4)
        memberships = fuzzify(avg_pred)
        print(f"[INFO] Average prediction: {avg_pred}")
        current_status = max(memberships, key=memberships.get)
        print(f"[INFO] Final status: {current_status}")
    else:
        current_status = "focused"
        avg_pred = 0
        print("[INFO] Classification: Focused")

@app.route('/')
def home():
    return "Flask server is running."

@app.route("/upload", methods=["GET", "POST"])
def upload():
  if request.method == "POST":
        print("Received a POST request!")
        if 'video' not in request.files:
            print("[ERROR] No video file in request.")
            return jsonify({'error': 'No video uploaded'}), 400

        video = request.files['video']
        video_path = os.path.join(app.config['UPLOAD_FOLDER'], 'latest_video.webm')
        mp4_path = os.path.join(app.config['UPLOAD_FOLDER'], 'converted.mp4')
        video.save(video_path)
        print(f"[INFO] Video saved to {video_path}")

        global current_status, avg_pred
        current_status = "focused"
        avg_pred = 0

        if not convert_webm_to_mp4(video_path, mp4_path):
            return jsonify({'error': 'Video conversion failed'}), 500

        process_video(mp4_path)

        return jsonify({
            'message': 'Video processed successfully',
            'result': current_status,
            'pred': float(round(avg_pred, 4))
        })

  elif request.method == "GET":
        return "Upload endpoint ready to receive POST requests!"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


Writing app.py


In [ ]:
!python3 app.py &>/dev/null&

In [ ]:
!./ngrok authtoken 2qYswpecIJhKSdstSAKprD4SsCf_6hP15b2RiSoHcXKtMdv6c

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import subprocess
import json
import time

ngrok_process = subprocess.Popen(['./ngrok', 'http', '5000'])

time.sleep(10)

!curl -s http://localhost:4040/api/tunnels > tunnels.json

try:
    with open("tunnels.json") as f:
        data = json.load(f)
    ngrok_url = data['tunnels'][0]['public_url']
    print(f"\n🚀 Your Flask app is publicly available at: {ngrok_url}\n")
except json.JSONDecodeError:
    print("Error: Could not retrieve ngrok URL. Make sure ngrok is running and the tunnel is established.")
    ngrok_process.terminate()


🚀 Your Flask app is publicly available at: https://b308-34-48-85-241.ngrok-free.app



To install ngrok

In [ ]:
!python app.py

2025-04-17 06:30:54.010254: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744871454.035251   26488 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744871454.043649   26488 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-17 06:30:54.070518: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[INFO] Loading model...
2025-04-17 06:30:58.724013: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed 

In [ ]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz

--2025-04-17 06:26:56--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 99.83.220.108, 35.71.179.82, 13.248.244.96, ...
Connecting to bin.equinox.io (bin.equinox.io)|99.83.220.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9395172 (9.0M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   8.96M  --.-KB/s    in 0.06s   

2025-04-17 06:26:56 (149 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [9395172/9395172]



In [ ]:
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz

ngrok
